In [8]:
#Loading CSV file
import pandas as pd

# Read the CSV file
df = pd.read_csv('mental_health.csv')

# Display the first few rows
print("\nFirst few rows of the dataset:")
print(df.head())

# Display basic information about the dataset
print("\nDataset Info:")
print(f"Total rows: {len(df)}")
print(f"Total columns: {len(df.columns)}")


First few rows of the dataset:
   Are you self-employed?  \
0                       0   
1                       0   
2                       0   
3                       1   
4                       0   

  How many employees does your company or organization have?  \
0                                             26-100           
1                                               6-25           
2                                               6-25           
3                                                NaN           
4                                               6-25           

   Is your employer primarily a tech company/organization?  \
0                                                1.0         
1                                                1.0         
2                                                1.0         
3                                                NaN         
4                                                0.0         

   Is your primary role within your

In [6]:
#Parsing the data
import csv
from collections import defaultdict

def parse_mental_health_data(filename):
    # Dictionary to store our parsed data
    data = defaultdict(list)
    
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            # Create CSV reader
            csv_reader = csv.DictReader(file)
            
            # Get headers
            headers = csv_reader.fieldnames
            print("Column headers:", headers)
            
            # Parse each row
            for row in csv_reader:
                # Convert numeric strings to appropriate types
                parsed_row = {}
                for key, value in row.items():
                    # Try to convert to float if it looks like a number
                    if value.replace('.', '').isdigit():
                        parsed_row[key] = float(value)
                    # Handle empty strings
                    elif value.strip() == '':
                        parsed_row[key] = None
                    else:
                        parsed_row[key] = value
                
                # Store parsed row data in respective columns
                for key, value in parsed_row.items():
                    data[key].append(value)
                    
        # Print summary statistics
        print("\nDataset Summary:")
        print(f"Total rows: {len(next(iter(data.values())))}")
        print("\nSample of first 5 rows:")
        for i in range(min(5, len(next(iter(data.values()))))):
            row_data = {key: values[i] for key, values in data.items()}
            print(f"Row {i+1}: {row_data}")
            
        return data
            
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found")
        return None
    except Exception as e:
        print(f"Error parsing file: {str(e)}")
        return None

# Parse the cleaned dataset
data = parse_mental_health_data('mental_health.csv')




Column headers: ['Are you self-employed?', 'How many employees does your company or organization have?', 'Is your employer primarily a tech company/organization?', 'Is your primary role within your company related to tech/IT?', 'Does your employer provide mental health benefits as part of healthcare coverage?', 'Do you know the options for mental health care available under your employer-provided coverage?', 'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?', 'Does your employer offer resources to learn more about mental health concerns and options for seeking help?', 'Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?', 'If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:', 'Do you think that discussing a mental health disorder with your employer would ha

In [12]:
#Normalize the data
import sqlite3
from collections import defaultdict

def create_database_schema():
    """Create normalized database schema for mental health survey data"""
    conn = sqlite3.connect('mental_health_survey.db')
    cursor = conn.cursor()
    
    # [Previous CREATE TABLE statements remain the same...]
    # Schema creation code remains unchanged as it's a one-time operation
    
    return conn

def normalize_and_load_data(data):
    """Normalize and load the parsed data into the database using batch inserts"""
    conn = create_database_schema()
    cursor = conn.cursor()
    
    # Prepare data for batch insertion
    total_records = len(next(iter(data.values())))
    
    # Prepare employees data
    employees_data = [
        (
            data['Are you self-employed?'][i],
            data['How many employees does your company or organization have?'][i],
            data['Is your employer primarily a tech company/organization?'][i],
            data['Is your primary role within your company related to tech/IT?'][i],
            data['What is your age?'][i],
            data['What is your gender?'][i],
            data['What country do you live in?'][i],
            data['What US state or territory do you live in?'][i],
            data['What country do you work in?'][i],
            data['What US state or territory do you work in?'][i],
            data['Which of the following best describes your work position?'][i],
            data['Do you work remotely?'][i]
        )
        for i in range(total_records)
    ]
    
    # Insert employees and get IDs
    cursor.executemany('''
    INSERT INTO employees (
        is_self_employed, company_size, is_tech_company, is_tech_role,
        age, gender, country, state, work_country, work_state,
        position, is_remote
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', employees_data)
    
    # Get the ID of the first inserted employee
    first_employee_id = cursor.lastrowid - total_records + 1
    
    # Prepare benefits data
    benefits_data = [
        (
            first_employee_id + i,
            data['Does your employer provide mental health benefits as part of healthcare coverage?'][i],
            data['Do you know the options for mental health care available under your employer-provided coverage?'][i],
            data['Do you have medical coverage (private insurance or state-provided) which includes treatment of \xa0mental health issues?'][i]
        )
        for i in range(total_records)
    ]
    
    # Insert benefits data
    cursor.executemany('''
    INSERT INTO benefits (
        employee_id, has_mental_health_benefits, knows_benefits_options,
        has_mental_health_coverage
    ) VALUES (?, ?, ?, ?)
    ''', benefits_data)
    
    # Prepare workplace environment data
    workplace_env_data = [
        (
            first_employee_id + i,
            data['Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?'][i],
            data['Does your employer offer resources to learn more about mental health concerns and options for seeking help?'][i],
            data['Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?'][i],
            data['If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:'][i],
            data['Do you feel that your employer takes mental health as seriously as physical health?'][i],
            data['Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?'][i]
        )
        for i in range(total_records)
    ]
    
    cursor.executemany('''
    INSERT INTO workplace_environment (
        employee_id, formal_discussion, resources_available,
        anonymity_protected, leave_request_ease, mental_health_seriously,
        observed_negative_consequences
    ) VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', workplace_env_data)
    
    # Prepare previous employment data
    prev_emp_data = [
        (
            first_employee_id + i,
            data['Do you have previous employers?'][i],
            data['Have your previous employers provided mental health benefits?'][i],
            data['Were you aware of the options for mental health care provided by your previous employers?'][i],
            data['Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?'][i],
            data['Did your previous employers provide resources to learn more about mental health issues and how to seek help?'][i],
            data['Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?'][i],
            data['Do you think that discussing a mental health disorder with previous employers would have negative consequences?'][i],
            data['Do you think that discussing a physical health issue with previous employers would have negative consequences?'][i],
            data['Would you have been willing to discuss a mental health issue with your previous co-workers?'][i],
            data['Would you have been willing to discuss a mental health issue with your direct supervisor(s)?'][i],
            data['Did you feel that your previous employers took mental health as seriously as physical health?'][i],
            data['Did you hear of or observe negative consequences for co-workers with mental health issues in your previous workplaces?'][i]
        )
        for i in range(total_records)
    ]
    
    cursor.executemany('''
    INSERT INTO previous_employment (
        employee_id, has_previous_employers, provided_mental_benefits,
        aware_of_benefits, formal_discussion, resources_available,
        anonymity_protected, mental_health_consequences,
        physical_health_consequences, comfortable_discussing_coworkers,
        comfortable_discussing_supervisor, mental_health_seriously,
        observed_negative_consequences
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', prev_emp_data)
    
    # Prepare mental health history data
    history_data = [
        (
            first_employee_id + i,
            data['Do you have a family history of mental illness?'][i],
            data['Have you had a mental health disorder in the past?'][i],
            data['Do you currently have a mental health disorder?'][i],
            data['If yes, what condition(s) have you been diagnosed with?'][i],
            data['If maybe, what condition(s) do you believe you have?'][i],
            data['Have you been diagnosed with a mental health condition by a medical professional?'][i],
            data['Have you ever sought treatment for a mental health issue from a mental health professional?'][i]
        )
        for i in range(total_records)
    ]
    
    cursor.executemany('''
    INSERT INTO mental_health_history (
        employee_id, has_family_history, past_disorder,
        current_disorder, diagnosed_conditions, believed_conditions,
        professional_diagnosis, sought_treatment
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', history_data)
    
    # Prepare work impact data
    impact_data = [
        (
            first_employee_id + i,
            data['Do you believe your productivity is ever affected by a mental health issue?'][i],
            data['If yes, what percentage of your work time (time performing primary or secondary job functions) is affected by a mental health issue?'][i],
            data['If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?'][i],
            data['If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?'][i]
        )
        for i in range(total_records)
    ]
    
    cursor.executemany('''
    INSERT INTO work_impact (
        employee_id, productivity_affected, affected_work_time,
        interferes_when_treated, interferes_when_untreated
    ) VALUES (?, ?, ?, ?, ?)
    ''', impact_data)
    
    conn.commit()
    conn.close()

# Example usage:
# normalize_and_load_data(data)

In [11]:
import os
print(os.getcwd())  # This will show your current working directory

C:\Users\ASHISH


In [13]:
import sqlite3
import os
from collections import defaultdict
import shutil

def create_database_schema():
    """Create normalized database schema for mental health survey data"""
    # Create database in a temporary file first to avoid corrupting existing data
    temp_db = 'temp_mental_health_survey.db'
    conn = sqlite3.connect(temp_db)
    cursor = conn.cursor()
    
    # [Previous CREATE TABLE statements remain the same...]
    
    return conn, temp_db

def save_database(temp_db, final_db='mental_health_survey.db'):
    """Safely save the database to its final location"""
    try:
        # If final database exists, create a backup
        if os.path.exists(final_db):
            backup_db = f"{final_db}.backup"
            shutil.copy2(final_db, backup_db)
            print(f"Created backup at: {backup_db}")
        
        # Move temporary database to final location
        shutil.move(temp_db, final_db)
        print(f"Database saved successfully at: {final_db}")
        
        # Verify the database is valid
        conn = sqlite3.connect(final_db)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
        tables = cursor.fetchall()
        print(f"Database contains {len(tables)} tables")
        conn.close()
        
        return True
        
    except Exception as e:
        print(f"Error saving database: {str(e)}")
        # If error occurs and backup exists, restore from backup
        if os.path.exists(backup_db):
            shutil.copy2(backup_db, final_db)
            print("Restored from backup due to save error")
        return False

def normalize_and_load_data(data, final_db='mental_health_survey.db'):
    """Normalize and load the parsed data into the database using batch inserts"""
    conn, temp_db = create_database_schema()
    cursor = conn.cursor()
    
    try:
        # [Previous data preparation and insertion code remains the same...]
        
        # Commit all changes
        conn.commit()
        conn.close()
        
        # Save the database to its final location
        if save_database(temp_db, final_db):
            print("Database creation and save completed successfully")
            
            # Clean up backup if it exists
            backup_db = f"{final_db}.backup"
            if os.path.exists(backup_db):
                os.remove(backup_db)
        else:
            print("Error during database save process")
            
    except Exception as e:
        print(f"Error during data loading: {str(e)}")
        conn.rollback()
        conn.close()
        
        # Clean up temporary file if it exists
        if os.path.exists(temp_db):
            os.remove(temp_db)
            
        raise e

# Example usage:
# normalize_and_load_data(data, 'mental_health_survey.db')

In [15]:
#Querying the necessary data
import sqlite3
import pandas as pd

def query_mental_health_data():
    """Query and analyze mental health survey data"""
    conn = sqlite3.connect('mental_health_survey.db')
    
    # Query 1: Basic employee demographics with benefits information
    demographics_query = """
    SELECT 
        e.employee_id,
        e.age,
        e.gender,
        e.country,
        e.company_size,
        e.is_tech_company,
        b.has_mental_health_benefits,
        b.knows_benefits_options
    FROM employees e
    LEFT JOIN benefits b ON e.employee_id = b.employee_id
    """
    demographics_df = pd.read_sql_query(demographics_query, conn)
    print("\nEmployee Demographics and Benefits:")
    print(demographics_df.head())
    
    # Query 2: Workplace environment and mental health support
    workplace_query = """
    SELECT 
        e.employee_id,
        e.company_size,
        we.formal_discussion,
        we.resources_available,
        we.leave_request_ease,
        we.mental_health_seriously,
        wi.productivity_affected,
        wi.affected_work_time
    FROM employees e
    LEFT JOIN workplace_environment we ON e.employee_id = we.employee_id
    LEFT JOIN work_impact wi ON e.employee_id = wi.employee_id
    """
    workplace_df = pd.read_sql_query(workplace_query, conn)
    print("\nWorkplace Environment Analysis:")
    print(workplace_df.head())
    
    # Query 3: Mental health history and work impact
    health_impact_query = """
    SELECT 
        e.employee_id,
        e.is_tech_company,
        mh.has_family_history,
        mh.current_disorder,
        mh.sought_treatment,
        wi.productivity_affected,
        wi.interferes_when_treated,
        wi.interferes_when_untreated
    FROM employees e
    LEFT JOIN mental_health_history mh ON e.employee_id = mh.employee_id
    LEFT JOIN work_impact wi ON e.employee_id = wi.employee_id
    """
    health_impact_df = pd.read_sql_query(health_impact_query, conn)
    print("\nMental Health and Work Impact Analysis:")
    print(health_impact_df.head())
    
    # Query 4: Previous employment and current workplace comparison
    employment_comparison_query = """
    SELECT 
        e.employee_id,
        we.mental_health_seriously as current_employer_seriousness,
        pe.mental_health_seriously as previous_employer_seriousness,
        we.formal_discussion as current_formal_discussion,
        pe.formal_discussion as previous_formal_discussion,
        we.resources_available as current_resources,
        pe.resources_available as previous_resources
    FROM employees e
    LEFT JOIN workplace_environment we ON e.employee_id = we.employee_id
    LEFT JOIN previous_employment pe ON e.employee_id = pe.employee_id
    WHERE pe.has_previous_employers = 1
    """
    employment_comparison_df = pd.read_sql_query(employment_comparison_query, conn)
    print("\nCurrent vs Previous Employment Comparison:")
    print(employment_comparison_df.head())
    
    # Query 5: Communication preferences and disclosure impact
    communication_query = """
    SELECT 
        e.employee_id,
        e.position,
        cp.comfortable_coworkers,
        cp.comfortable_supervisor,
        cp.reveal_clients,
        cp.client_impact,
        ia.career_impact_concern,
        ia.sharing_willingness
    FROM employees e
    LEFT JOIN communication_preferences cp ON e.employee_id = cp.employee_id
    LEFT JOIN interview_attitudes ia ON e.employee_id = ia.employee_id
    """
    communication_df = pd.read_sql_query(communication_query, conn)
    print("\nCommunication and Disclosure Analysis:")
    print(communication_df.head())
    
    # Close the connection
    conn.close()
    
    return {
        'demographics': demographics_df,
        'workplace': workplace_df,
        'health_impact': health_impact_df,
        'employment_comparison': employment_comparison_df,
        'communication': communication_df
    }

# Function to analyze specific aspects of the data
def analyze_tech_vs_non_tech():
    """Compare mental health support in tech vs non-tech companies"""
    conn = sqlite3.connect('mental_health_survey.db')
    
    tech_comparison_query = """
    SELECT 
        e.is_tech_company,
        b.has_mental_health_benefits,
        we.mental_health_seriously,
        we.resources_available,
        COUNT(*) as company_count
    FROM employees e
    LEFT JOIN benefits b ON e.employee_id = b.employee_id
    LEFT JOIN workplace_environment we ON e.employee_id = we.employee_id
    GROUP BY e.is_tech_company
    """
    
    tech_comparison_df = pd.read_sql_query(tech_comparison_query, conn)
    conn.close()
    return tech_comparison_df

# Example usage:
# dataframes = query_mental_health_data()
# tech_comparison = analyze_tech_vs_non_tech()